# Basic linear modeling

In this exercise we will run a simple regression on all voxels in a 4D
FMRI image `ds114_sub009_t2r1.nii`:

In [ ]:
#: Import some standard librares
import numpy as np
# Print to 4 DP
np.set_printoptions(precision=4)
import numpy.linalg as npl
import matplotlib.pyplot as plt

# Set default imshow parameters
plt.rcParams['image.cmap'] = 'gray'

import nibabel as nib

import nipraxis

In [ ]:
# Fetch and load the data file.
data_fname = nipraxis.fetch_file('ds114_sub009_t2r1.nii')
img = nib.load(data_fname)
data = img.get_fdata()
# Knock off first four volumes.
data = data[..., 4:]

We make the design matrix from the convolved regressor from [Convolving with
the hemodyamic response
function](https://textbook.nipraxis.org/convolution_background.html):

In [ ]:
# Load the pre-written convolved time course
conv_fname = nipraxis.fetch_file('ds114_sub009_t2r1_conv.txt')
# Knock off the first four elements
convolved = np.loadtxt(conv_fname)[4:]
plt.plot(convolved)

Now compile the design matrix for a simple regression, where the regressor (independent variable) is the convolved time-course.

In [ ]:
#- Compile the design matrix
#- First column is convolved regressor
#- Second column all ones
#- Hint: investigate "aspect" keyword to ``plt.imshow`` for a nice
#- looking image.
# Show an image of the resulting design matrix.
plt.imshow(design, aspect=0.1)

In [ ]:
assert design.shape == (len(convolved), 2)
assert np.allclose(np.mean(design, axis=0), [np.mean(convolved), 1])

Next reshape the voxel data to two dimensions, where the first dimension is
voxels, and the second dimension is time (volume).  Therefore there should be
one column for each volume (time-point), and one row for each voxel.  Then transpose to get a time by voxels array.

You may want to have another look at [reshaping 4D to 2D](https://textbook.nipraxis.org/reshape_and_4d.html).


In [ ]:
#- Reshape the 4D data to voxel by time 2D, and transpose to time by voxels.
data_2d = ...
# Show the resulting shape
data_2d.shape

In [ ]:
assert len(data_2d.shape) == 2, 'Did you reshape to 2D?'
assert data_2d.shape[0] == data.shape[-1], \
    'Did you transpose to time by voxels?'
assert data_2d.shape[1] == np.prod(data.shape[:3])
assert np.all(data_2d[:, 0] == data[0, 0, 0, :]), 'First col is first voxel'
assert np.all(data_2d[:, -1] == data[-1, -1, -1, :]), 'Last col is last voxel'

Next estimate the betas for the design, by matrix-multiplying on the left with the pseudo-inverse of the design matrix.

In [ ]:
#- Calculate the pseudoinverse of the design
#- Apply to time by voxel array to get betas
betas_2d = ...
# Show the resulting shape
betas_2d.shape

In [ ]:
assert betas_2d.shape == (2, np.prod(data.shape[:3]))

You now want to reshape the betas into a four dimensional array, with the
first three axes being voxel axes, and the last, of length 2, being the
parameters.  So the resulting 4D array will be a stack of two 3D arrays, where the first 3D array has the parameters from the first column of the design matrix, and the second has the parameters from the second column.

In [ ]:
#- Transpose betas to give voxels by 2 array
#- Reshape into 4D array, with same 3D shape as original data,
#- last dimension length 2
betas_4d = ...
# Show the resulting shape
betas_4d.shape

In [ ]:
assert betas_4d.shape[-1] == 2, 'Did you do any necessary transpose?'
assert betas_4d.shape == data.shape[:3] + (2,)
assert np.all(betas_4d[0, 0, 0, :] == betas_2d[:, 0]), 'First voxel is first col'
assert np.all(betas_4d[-1, -1, -1, :] == betas_2d[:, -1]), 'Last voxel is last col'

In [ ]:
#- Show the middle slice from the first beta volume

In [ ]:
#- Show the middle slice from the second beta volume